In [1]:
import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import numpy as np 
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
from tqdm import tqdm 

In [4]:
def PSNR(im_gt,im_output):
    mse = np.mean((im_gt-im_output)**2)
    if mse ==0:
        psnr = float('inf')
    else:
        psnr = 10* np.log10(255.0**2/mse)
    return psnr

In [9]:
import os
os.chdir('/data/public/PATH-DT-MSU/SR_patches_10x_20x_40x/')
netscale = 4

In [12]:
iter_list = list(range(5000, 100001, 5000))
model_name_list = ['net_g_'+ str(model_name) +'.pth' for model_name in iter_list]
# iter_list
# print(model_name_list)
model_dir = '/home/zsun/Forme/Real-ESRGAN-master-2/experiments/finetune_RealESRGANx4plus_400k_pairdata/models'
# model_dir = '/home/zsun/Forme/Real-ESRGAN-master-2/fine-tune_1/models'
mean_psnr_list = np.zeros_like(iter_list,dtype=np.float32)
print(mean_psnr_list)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
info_path = '/home/zsun/Forme/Real-ESRGAN-master-2/meta_test_double.txt'
with open(info_path,'r') as f :
    i = 0
    for model_name in model_name_list:
        model_path = os.path.join(model_dir,model_name)  
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
        upsampler = RealESRGANer(
            scale=netscale,
            model_path=model_path,
            dni_weight=None,
            model=model,
            tile=0,
            tile_pad=10,
            pre_pad=0,
            half=not False,
            gpu_id=None)
        print(f"model {model_name} is created")
        sum_psnr = 0
        img_paths = [line.strip() for line in f]
        for pair_path in tqdm(img_paths,total = len(img_paths)):
            ref_path = pair_path.split(', ')[0]
            lr_path = pair_path.split(', ')[1]
            # print(ref_path,lr_path)
            im_ref = cv2.imread(ref_path)
            im_lr = cv2.imread(lr_path)
            try:
                im_output,_ = upsampler.enhance(im_lr,outscale=4)
            except RuntimeError as error:
                print('Error', error)
                print('If you encounter CUDA out of memory, try to set --tile with a smaller number.')
            psnr = PSNR(im_gt=im_ref,im_output=im_output)
            # print(psnr)
            sum_psnr +=psnr
            # break
        mean_psnr_list[i] = sum_psnr / len(img_paths)
        print(mean_psnr_list[i])
        i = i+1
        break
        


model net_g_5000.pth is created


 27%|██▋       | 929/3457 [08:44<23:43,  1.78it/s]